In [1]:
import pandas as pd
from PIL import Image
import numpy as np
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

In [2]:
#Procesado de img (a 16x16 para agilizar el entrenamiento)
def load_and_process_image(image_path):
    image = Image.open(image_path)
    image = image.resize((16, 16)) # Cambia el tamaño de la imagen a 64x64 para obtener más características HOG
    image = image.convert('L') # Convertir a escala de grises ya que HOG se calcula sobre imágenes en escala de grises
    return np.array(image)

In [3]:
#hog
def extract_features(images):
    hog_features = []
    for image in images:
        fd = hog(image, orientations=8, pixels_per_cell=(16,16), cells_per_block=(1, 1))
        hog_features.append(fd)
    return np.array(hog_features)


In [4]:
#Csv
print("Cargando datos de los CSV...")
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')
val_data = pd.read_csv('val_data.csv')


Cargando datos de los CSV...


In [5]:
#Carga+procesado+obtencion de etiquetas
print("procesando imágenes...")
X_train = np.array([load_and_process_image(img_path) for img_path in train_data['path']])
y_train = train_data['label']
X_test = np.array([load_and_process_image(img_path) for img_path in test_data['path']])
y_test = test_data['label']
X_val = np.array([load_and_process_image(img_path) for img_path in val_data['path']])
y_val = val_data['label']
print("imágenes procesadas con éxito.")


procesando imágenes...
imágenes procesadas con éxito.


In [6]:
# características HOG
print("Extrayendo HOG features...")
X_train_hog = extract_features(X_train)
X_test_hog = extract_features(X_test)
X_val_hog = extract_features(X_val)
print("HOG extraídas con éxito.")

Extrayendo HOG features...
HOG extraídas con éxito.


In [7]:
#clasificador SVM
clf = svm.SVC()

In [8]:
# Entrenamiento
print("Entrenando el modelo SVM...")
clf.fit(X_train_hog, y_train)
print("Al fin, entrenado.")

Entrenando el modelo SVM...
Al fin, entrenado.


In [9]:
# predicciones
print("Realizando predicciones en test_data...")
y_pred_test = clf.predict(X_test_hog)
print("Realizando predicciones en val_data...")
y_pred_val = clf.predict(X_val_hog)


Realizando predicciones en test_data...
Realizando predicciones en val_data...


In [10]:
# Evaluacion
print("Evaluando modelo...")
print('\nAccuracy en el test: ',accuracy_score(y_test, y_pred_test))
print('\n',classification_report(y_test, y_pred_test))
print('\nAccuracy en validación: ',accuracy_score(y_val, y_pred_val))
print('\n',classification_report(y_val, y_pred_val))


Evaluando modelo...

Accuracy en el test:  0.7179129933459848

               precision    recall  f1-score   support

           0       0.72      1.00      0.84     29886
           1       0.00      0.00      0.00     11743

    accuracy                           0.72     41629
   macro avg       0.36      0.50      0.42     41629
weighted avg       0.52      0.72      0.60     41629


Accuracy en validación:  0.7143097360013452

               precision    recall  f1-score   support

           0       0.71      1.00      0.83     29736
           1       0.00      0.00      0.00     11893

    accuracy                           0.71     41629
   macro avg       0.36      0.50      0.42     41629
weighted avg       0.51      0.71      0.60     41629



C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\lib\site-packages\sklearn\metr

In [11]:
# Guardar el modelo
from joblib import dump
dump(clf, 'svm_model.joblib') 
print("Modelo guardado con éxito.")

Modelo guardado con éxito.
